# Prediksi Harga Pangan dengan GNN - Versi Peningkatan

Notebook ini bertujuan untuk memprediksi harga pangan menggunakan Graph Neural Networks (GNN) dengan menerapkan berbagai peningkatan berdasarkan praktik terbaik.

## 1. Impor Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 2. Memuat dan Pra-pemrosesan Data Awal
(Asumsikan data dimuat ke dalam DataFrame `df_harga_pangan`)

In [ ]:
# Placeholder: Muat data Anda di sini
# df_harga_pangan = pd.read_csv('path_to_your_data.csv', parse_dates=['tanggal'])
# df_harga_pangan = df_harga_pangan.pivot(index='tanggal', columns='provinsi', values='harga')
# df_harga_pangan.ffill(inplace=True) # Isi missing values jika ada

# Contoh data dummy jika tidak ada data nyata
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
n_provinces = 34
data_dummy = np.random.rand(len(dates), n_provinces) * 10000 + 5000
df_harga_pangan = pd.DataFrame(data_dummy, index=dates, columns=[f'Provinsi_{i+1}' for i in range(n_provinces)])

print("Shape data awal:", df_harga_pangan.shape)
df_harga_pangan.head()

## 3. Peningkatan Fitur (Feature Engineering) (Poin 6)
Menambahkan fitur temporal seperti hari dalam seminggu, bulan, dll.

In [ ]:
def enhance_features(df):
    df_enhanced = df.copy()
    if isinstance(df_enhanced.index, pd.DatetimeIndex):
        df_enhanced['day_of_week'] = df_enhanced.index.dayofweek
        df_enhanced['month'] = df_enhanced.index.month
        df_enhanced['day_of_year'] = df_enhanced.index.dayofyear
        df_enhanced['week_of_year'] = df_enhanced.index.isocalendar().week.astype(int)
    # Tambahkan fitur lain jika perlu, misal: lagged features, rolling statistics
    return df_enhanced

# Terapkan enhance_features sebelum membuat sekuens
# Jika df_harga_pangan adalah harga saja, kita perlu struktur yang berbeda untuk fitur tambahan
# Untuk GNN, fitur biasanya per node. Jika fitur temporal ini global, cara integrasinya perlu dipikirkan.
# Untuk saat ini, kita asumsikan fitur ini akan digabungkan ke data input node.

# Mari kita asumsikan df_harga_pangan adalah DataFrame utama yang akan diproses lebih lanjut
# Fitur tambahan ini bisa digunakan saat membuat snapshot X untuk GNN
# df_harga_pangan_featured = enhance_features(df_harga_pangan.reset_index().set_index('tanggal')) # Contoh jika tanggal adalah index
# print("Data dengan fitur tambahan:")
# print(df_harga_pangan_featured.head())

## 4. Normalisasi Data (Poin 4)
Normalisasi per provinsi. Menggunakan `fit_transform` pada data latih dan `transform` pada data validasi/tes.

In [ ]:
# Data splitting (Poin 1 & 9: Menggunakan 20% akhir untuk tes, 20% sebelumnya untuk validasi)
total_samples = len(df_harga_pangan)
test_size_ratio = 0.20
val_size_ratio = 0.20 # Dari sisa data setelah test

test_split_idx = int(total_samples * (1 - test_size_ratio))
val_split_idx = int(test_split_idx * (1 - val_size_ratio / (1-test_size_ratio) )) # val_size dari sisa train+val

df_train_val = df_harga_pangan.iloc[:test_split_idx]
df_test = df_harga_pangan.iloc[test_split_idx:]

df_train = df_train_val.iloc[:val_split_idx]
df_val = df_train_val.iloc[val_split_idx:]

print(f"Ukuran data Latih: {df_train.shape}")
print(f"Ukuran data Validasi: {df_val.shape}")
print(f"Ukuran data Tes: {df_test.shape}")

# Normalisasi
scalers = {}
df_train_scaled = pd.DataFrame(index=df_train.index)
df_val_scaled = pd.DataFrame(index=df_val.index)
df_test_scaled = pd.DataFrame(index=df_test.index)

for col in df_train.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_train_scaled[col] = scaler.fit_transform(df_train[[col]])
    df_val_scaled[col] = scaler.transform(df_val[[col]])
    df_test_scaled[col] = scaler.transform(df_test[[col]])
    scalers[col] = scaler

print("\nData latih setelah normalisasi (contoh):")
print(df_train_scaled.head())

## 5. Membuat Sekuens untuk GNN
Membuat sekuens input (X) dan target (y) untuk model GNN.
Fitur yang dihasilkan oleh `enhance_features` harus diintegrasikan di sini saat membuat `X`.

In [ ]:
def create_sequences(input_data, target_data, window_size, prediction_horizon):
    X, y = [], []
    # Asumsikan input_data dan target_data adalah pd.DataFrame dengan kolom provinsi
    # dan index adalah waktu.
    
    # Untuk fitur tambahan dari enhance_features:
    # Jika enhance_features menghasilkan kolom baru di input_data, itu akan otomatis masuk.
    # Jika fitur tambahan bersifat global (misal, hari libur), perlu cara lain untuk memasukkannya.
    # Saat ini, kita asumsikan fitur node adalah harga historis + fitur dari enhance_features jika ada.
    
    # Contoh sederhana: fitur node hanya harga historis
    # Jika ingin fitur dari enhance_features, pastikan input_data sudah mengandungnya.
    # Misal, jika input_data adalah df_harga_pangan_featured.
    
    num_nodes = input_data.shape[1] # Jumlah provinsi/node
    
    for i in range(len(input_data) - window_size - prediction_horizon + 1):
        # Fitur X: (window_size, num_nodes * num_features_per_node)
        # Jika hanya harga: (window_size, num_nodes)
        # Jika harga + fitur lain (misal, dari enhance_features):
        #   perlu reshape X_slice agar sesuai.
        #   Contoh: X_slice = input_data.iloc[i:i + window_size].values # (window_size, num_nodes * num_base_features)
        #   Jika ada fitur tambahan, misal 'day_of_week', 'month' yang sama untuk semua node pada satu waktu:
        #   additional_feats = enhanced_features_df.iloc[i:i+window_size][['day_of_week', 'month']].values
        #   X_slice = np.concatenate([X_slice, np.tile(additional_feats, (1, num_nodes))], axis=1) # Ini perlu penyesuaian
        
        # Untuk GNN, X biasanya (num_nodes, num_features_per_node) per snapshot.
        # Jadi, kita buat list of snapshots.
        
        # Snapshot X: (num_nodes, window_size) - setiap node memiliki fitur berupa time series window_size hari terakhir.
        # Atau, jika fitur lain ditambahkan: (num_nodes, window_size + num_additional_features)
        
        # Versi sederhana: X adalah harga selama window_size untuk semua provinsi
        # Target y adalah harga pada prediction_horizon hari ke depan untuk semua provinsi
        
        # Untuk GNN, kita perlu satu snapshot X (N, F) dan y (N, T_out) atau (N*T_out)
        # Di sini, kita akan buat dataset di mana setiap item adalah satu graph (snapshot)
        # X_snapshot: (num_nodes, window_size)
        # y_snapshot: (num_nodes, 1) # Prediksi 1 hari ke depan
        
        # Ambil data untuk window saat ini
        window_data = input_data.iloc[i : i + window_size].values # Shape: (window_size, num_nodes)
        
        # Transpose agar menjadi (num_nodes, window_size) -> ini akan jadi fitur node
        X_snapshot_features = window_data.T # Shape: (num_nodes, window_size)
        
        # Target: harga pada hari ke-(i + window_size + prediction_horizon - 1)
        # Shape: (num_nodes, 1)
        y_snapshot_target = target_data.iloc[i + window_size + prediction_horizon - 1].values.reshape(-1, 1)
        
        X.append(X_snapshot_features)
        y.append(y_snapshot_target)
        
    return np.array(X), np.array(y)

window_size = 30 # 30 hari terakhir sebagai input
prediction_horizon = 1 # Prediksi 1 hari ke depan

# Menggunakan data yang sudah dinormalisasi
# Untuk fitur tambahan dari enhance_features, perlu digabungkan ke df_train_scaled, df_val_scaled, df_test_scaled
# sebelum memanggil create_sequences.
# Contoh:
# df_train_featured = enhance_features(df_train.reset_index().set_index('tanggal')) # Lakukan pada data asli sebelum scaling
# Kemudian scale fitur tambahan ini juga dan gabungkan dengan df_train_scaled
# Untuk GNN, fitur node bisa berupa [harga_hist_1, ..., harga_hist_30, day_of_week_t, month_t]
# Ini memerlukan modifikasi pada create_sequences untuk menangani fitur node yang lebih kompleks.

# Untuk kesederhanaan saat ini, fitur node hanya time series harga (window_size)
X_train, y_train = create_sequences(df_train_scaled, df_train_scaled, window_size, prediction_horizon)
X_val, y_val = create_sequences(df_val_scaled, df_val_scaled, window_size, prediction_horizon)
X_test, y_test = create_sequences(df_test_scaled, df_test_scaled, window_size, prediction_horizon)

print(f"Shape X_train: {X_train.shape}, y_train: {y_train.shape}") # (num_samples, num_nodes, num_node_features=window_size)
print(f"Shape X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"Shape X_test: {X_test.shape}, y_test: {y_test.shape}")

# Membuat adjacency matrix (contoh: fully connected graph antar provinsi)
num_nodes = df_harga_pangan.shape[1]
adj = np.ones((num_nodes, num_nodes)) - np.eye(num_nodes) # Terhubung ke semua kecuali diri sendiri
edge_index = torch.tensor(np.array(np.where(adj)), dtype=torch.long)

## 6. Membuat PyTorch Geometric Dataset dan DataLoader (Poin 3)
Menggunakan `shuffle=True` untuk DataLoader data latih.

In [ ]:
def create_graph_dataset(X_data, y_data, edge_idx):
    dataset = []
    for i in range(X_data.shape[0]):
        x = torch.tensor(X_data[i], dtype=torch.float) # Node features (num_nodes, num_node_features)
        y_sample = torch.tensor(y_data[i], dtype=torch.float) # Target (num_nodes, 1)
        data = Data(x=x, edge_index=edge_idx, y=y_sample)
        dataset.append(data)
    return dataset

train_dataset = create_graph_dataset(X_train, y_train, edge_index)
val_dataset = create_graph_dataset(X_val, y_val, edge_index)
test_dataset = create_graph_dataset(X_test, y_test, edge_index)

# Batch size akan di-tune oleh Optuna, ini nilai default
batch_size = 64 

# Poin 3: shuffle=True untuk training_loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Jumlah batch di train_loader: {len(train_loader)}")
print(f"Jumlah batch di val_loader: {len(val_loader)}")
print(f"Jumlah batch di test_loader: {len(test_loader)}")

## 7. Definisi Model GNN (Poin 2)
Mengurangi kompleksitas model: `hidden_channels` lebih kecil (misal 32-64), `num_layers` 2-3. Meningkatkan `dropout` (misal 0.3-0.5).

In [ ]:
class GCNPredictor(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_layers, dropout_rate, num_nodes):
        super(GCNPredictor, self).__init__()
        self.num_nodes = num_nodes
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(num_node_features, hidden_channels))
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        self.dropout_rate = dropout_rate
        # Output layer: dari hidden_channels ke 1 (prediksi harga per node)
        self.out_mlp = torch.nn.Linear(hidden_channels, 1) 

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)
            
        # x shape: (num_nodes_in_batch, hidden_channels)
        # Kita ingin prediksi per node, jadi langsung ke MLP
        x = self.out_mlp(x) # Output shape: (num_nodes_in_batch, 1)
        return x

# Contoh parameter (akan di-tune oleh Optuna)
# num_node_features adalah window_size jika hanya harga historis
# Jika ada fitur tambahan, ini akan berubah.
# X_train.shape[2] adalah num_node_features
# num_nodes adalah X_train.shape[1]

# model = GCNPredictor(num_node_features=X_train.shape[2], 
#                      hidden_channels=64, # Poin 2: Kurangi dari 128-256
#                      num_layers=2,       # Poin 2: Kurangi dari default
#                      dropout_rate=0.3,   # Poin 2: Tingkatkan dropout
#                      num_nodes=X_train.shape[1]) 
# print(model)

## 8. Fungsi Training dan Evaluasi

In [ ]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        # Target y ada di data.y, shape (num_nodes_in_batch, 1)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs # data.num_graphs adalah batch size
    return total_loss / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data in loader:
            out = model(data)
            loss = criterion(out, data.y)
            total_loss += loss.item() * data.num_graphs
            all_preds.append(out.cpu().numpy())
            all_targets.append(data.y.cpu().numpy())
            
    avg_loss = total_loss / len(loader.dataset)
    
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    
    # Reshape jika perlu, tergantung bentuk y (num_nodes, 1)
    # all_preds: (total_samples * num_nodes, 1)
    # all_targets: (total_samples * num_nodes, 1)
    
    # Untuk evaluasi RMSE/MAE pada skala asli, perlu inverse transform
    # Ini memerlukan scaler per provinsi dan perakitan kembali prediksi ke format (num_samples, num_nodes)
    # Untuk sementara, hitung metrik pada data ternormalisasi
    
    mse_scaled = mean_squared_error(all_targets, all_preds)
    mae_scaled = mean_absolute_error(all_targets, all_preds)
    
    return avg_loss, mse_scaled, mae_scaled, all_preds, all_targets

## 9. Hyperparameter Tuning dengan Optuna (Poin 8)
Memperluas rentang `batch_size`, `learning_rate`, `weight_decay`. Menambahkan `dropout` dan `num_units_mlp` (di sini `hidden_channels` dan `num_layers`).

In [ ]:
def objective(trial):
    # Hyperparameters to tune (Poin 8)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    hidden_channels = trial.suggest_categorical("hidden_channels", [32, 64, 128]) # Poin 2 & 8
    num_layers = trial.suggest_int("num_layers", 2, 3) # Poin 2 & 8
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5) # Poin 2 & 8
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True) # Poin 2 & 8
    current_batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128]) # Poin 8

    # Update DataLoaders dengan batch_size saat ini
    current_train_loader = DataLoader(train_dataset, batch_size=current_batch_size, shuffle=True)
    current_val_loader = DataLoader(val_dataset, batch_size=current_batch_size, shuffle=False)

    model = GCNPredictor(num_node_features=X_train.shape[2], 
                         hidden_channels=hidden_channels,
                         num_layers=num_layers,
                         dropout_rate=dropout_rate,
                         num_nodes=X_train.shape[1])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) # Poin 2: weight_decay
    
    # Poin 5: Loss training (mix MSE+MAE)
    # Untuk kesederhanaan, kita gunakan MSE. Bisa diganti dengan custom loss.
    criterion = torch.nn.MSELoss() 

    n_epochs = 40 # Sesuai info awal, bisa ditingkatkan
    best_val_loss = float('inf')
    epochs_no_improve = 0
    patience = 10 # Early stopping

    for epoch in range(n_epochs):
        train_loss = train_epoch(model, current_train_loader, optimizer, criterion)
        val_loss, _, _, _, _ = evaluate(model, current_val_loader, criterion)
        
        trial.report(val_loss, epoch) # Untuk pruning Optuna
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
    return best_val_loss

# Poin 8: Perluas jumlah trial
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50) # Tingkatkan n_trials dari 10

print("Best trial:")
trial = study.best_trial
print(f"  Value (Validation Loss): {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Latih model terbaik dengan parameter terbaik pada gabungan train+val, lalu evaluasi di test
best_params = trial.params
final_train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])
final_train_loader = DataLoader(final_train_dataset, batch_size=best_params['batch_size'], shuffle=True)

final_model = GCNPredictor(num_node_features=X_train.shape[2],
                           hidden_channels=best_params['hidden_channels'],
                           num_layers=best_params['num_layers'],
                           dropout_rate=best_params['dropout_rate'],
                           num_nodes=X_train.shape[1])

optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])
criterion = torch.nn.MSELoss()

print("\nTraining final model...")
# Latih untuk jumlah epoch yang sama atau lebih banyak, tanpa early stopping pada val set
# atau dengan early stopping pada subset train_val jika diinginkan
n_final_epochs = 60 # Bisa disesuaikan
for epoch in range(n_final_epochs):
    train_loss = train_epoch(final_model, final_train_loader, optimizer, criterion)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{n_final_epochs}, Train Loss: {train_loss:.6f}")

## 10. Evaluasi Model Terbaik pada Data Tes

In [ ]:
test_loss_scaled, test_mse_scaled, test_mae_scaled, preds_scaled, targets_scaled = evaluate(final_model, test_loader, criterion)

print(f"Test Loss (Scaled): {test_loss_scaled:.6f}")
print(f"Test MSE (Scaled): {test_mse_scaled:.6f}")
print(f"Test MAE (Scaled): {test_mae_scaled:.6f}")

# Inverse transform untuk mendapatkan metrik pada skala asli
# Preds_scaled dan targets_scaled memiliki shape (total_test_samples * num_nodes, 1)
# Perlu di-reshape ke (total_test_samples, num_nodes) sebelum inverse transform per kolom (provinsi)

num_test_samples = X_test.shape[0]
num_nodes = X_test.shape[1]

preds_scaled_reshaped = preds_scaled.reshape(num_test_samples, num_nodes)
targets_scaled_reshaped = targets_scaled.reshape(num_test_samples, num_nodes)

preds_original_scale = pd.DataFrame(columns=df_test_scaled.columns[:num_nodes]) # Sesuaikan jika kolom berbeda
targets_original_scale = pd.DataFrame(columns=df_test_scaled.columns[:num_nodes])

for i, col_name in enumerate(df_test_scaled.columns[:num_nodes]): # Ambil nama kolom dari df_test_scaled
    scaler = scalers[col_name]
    preds_original_scale[col_name] = scaler.inverse_transform(preds_scaled_reshaped[:, i].reshape(-1, 1)).flatten()
    targets_original_scale[col_name] = scaler.inverse_transform(targets_scaled_reshaped[:, i].reshape(-1, 1)).flatten()

# Hitung RMSE dan MAE pada skala asli
rmse_original = np.sqrt(mean_squared_error(targets_original_scale.values.flatten(), preds_original_scale.values.flatten()))
mae_original = mean_absolute_error(targets_original_scale.values.flatten(), preds_original_scale.values.flatten())

print(f"\nTest RMSE (Original Scale): {rmse_original:.2f}")
print(f"Test MAE (Original Scale): {mae_original:.2f}")

# Visualisasi contoh prediksi vs aktual untuk satu provinsi
province_to_plot = df_harga_pangan.columns[0] # Ambil provinsi pertama sebagai contoh
plt.figure(figsize=(15, 6))
plt.plot(targets_original_scale.index, targets_original_scale[province_to_plot], label='Actual Prices')
plt.plot(preds_original_scale.index, preds_original_scale[province_to_plot], label='Predicted Prices')
plt.title(f'Price Prediction for {province_to_plot}')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

## 11. Catatan Tambahan dan Pertimbangan Lanjutan

### Poin 5: Loss & Skala Output
- **Verifikasi Loss Weighting**: Jika menggunakan loss campuran (MSE+MAE), pastikan pembobotannya benar mengingat MSE bersifat kuadratik dan MAE linear.
- **Evaluasi Akhir**: Pastikan metrik evaluasi akhir (RMSE/MAE pada skala asli) tidak terpengaruh oleh bias dari proses scaling/unscaling. Periksa apakah `inverse_transform` dilakukan dengan benar.

### Poin 7: Arsitektur & Temporal Awareness
Model GCN saat ini memproses setiap snapshot secara independen. Untuk menangkap dependensi temporal antar snapshot secara lebih eksplisit, pertimbangkan:
- **GCN + RNN**: Menggabungkan output GCN dari setiap snapshot sebagai input ke lapisan RNN (GRU/LSTM).
- **Temporal GNN**: Menggunakan arsitektur GNN yang dirancang khusus untuk data sekuensial graf, seperti:
    - CTGCN (Temporal Graph Convolutional Network)
    - STGCN (Spatio-Temporal Graph Convolutional Network)
    - TGCN (Temporal Graph Convolutional Network - varian lain)
- **GCNConv + TimeConv**: Menggunakan lapisan konvolusi temporal (misalnya, Temporal Convolutional Network - TCN) setelah GCNConv untuk memproses dimensi waktu dari fitur node.
- **Graph Transformer**: Menggunakan mekanisme attention (misalnya, Transformer) untuk menangkap dependensi spasial dan temporal.

Implementasi arsitektur ini akan memerlukan perubahan signifikan pada definisi model dan mungkin juga pada cara data disiapkan.

### Poin Tambahan:
- **Walk-Forward Validation (Poin 9)**: Pemisahan data saat ini sudah lebih baik dengan mengambil blok waktu yang berurutan untuk train, val, dan test. Untuk evaluasi yang lebih robust pada data time series, pertimbangkan skema validasi walk-forward atau rolling window yang lebih ketat, di mana model dilatih ulang secara periodik dengan data baru.
- **Fitur Eksternal (Poin 6)**: Integrasi fitur eksternal (hari libur, peristiwa ekonomi, cuaca) dapat meningkatkan performa. Pastikan fitur ini diselaraskan dengan benar secara temporal dan diintegrasikan ke dalam fitur node atau sebagai input tambahan ke model.